In [5]:
# 1. Imports
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from typing import Optional
import json
import os
import re
import googlemaps
import random
from geopy.distance import distance as geopy_distance
from geopy.point import Point
import folium
from folium.plugins import AntPath
from folium.map import Icon
from IPython.display import display, clear_output
import requests
import time

# 2. إعداد مفاتيح API
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_MAPS_API_KEY"] = GOOGLE_API_KEY
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

# 3. استخراج وتحليل النص

def extract_locations_from_text(user_text: str) -> dict:
    prompt = f"""
أنت مساعد ذكي. المستخدم يتحدث باللهجة الأردنية ويعطيك جملة فيها موقعه الحالي والمكان الذي يريد الذهاب إليه.
مهمتك استخراج:
- نقطة البداية (start_location)
- نقطة الوجهة (end_location)
- الدولة (ثابتة: الأردن)

النص:
\"{user_text}\"

الرجاء إرجاع النتيجة بصيغة JSON فقط، مثل:
{{
    "start_location": "البوابة الجنوبية لجامعة اليرموك",
    "end_location": "جامعة العلوم والتكنولوجيا",
    "country": "الأردن"
}}
"""
    try:
        response = llm.invoke(prompt)
        result = eval(response.content)
    except Exception as e:
        result = {"error": f"لم يتمكن النموذج من استخراج المواقع بدقة. [{e}]"}
    return result

# 4. التعامل مع الأماكن المحفوظة

def load_saved_locations(file_path="C:\\Users\\anasf\\Desktop\\Okay\\Unihance Training\\Project 2\\Uber AgentChatBot\\saved_locations.json") -> dict:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"[خطأ] تعذر تحميل ملف الأماكن المحفوظة: {e}")
        return {}

def match_location_with_ai(user_place: str, saved_locations: dict, client) -> str:
    location_names = list(saved_locations.keys())
    locations_str = "\n".join(f"- {loc}" for loc in location_names)
    prompt = f"""
    المستخدم قال إنه يريد الذهاب إلى: "{user_place}"
    هذه قائمة بأسماء أماكن محفوظة موجودة في قاعدة البيانات:
    {locations_str}
    اختر أنسب مكان من القائمة يتطابق مع ما قاله المستخدم (بناءً على الفهم وليس التطابق الحرفي).
    إذا لم تجد تطابق واضح، قل فقط: TRUE
    أجب فقط باسم المكان من القائمة أو كلمة TRUE.
    """
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = client.invoke(prompt)
            choice = response.content.strip()
            break
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"[تحذير] حدث خطأ في الاتصال بـ OpenAI (محاولة {attempt+1}/{max_retries}): {e}. إعادة المحاولة...")
                time.sleep(2)
            else:
                print(f"[خطأ] فشل الاتصال بـ OpenAI بعد {max_retries} محاولات: {e}")
                return "لم أتمكن من تحديد الموقع بسبب مشكلة في خدمة الذكاء الاصطناعي. حاول لاحقًا."
    else:
        return "لم أتمكن من تحديد الموقع بدقة."
    if choice not in location_names and choice != "TRUE":
        return "لم أتمكن من تحديد الموقع بدقة."
    if choice == "TRUE":
        for key in saved_locations:
            if key in user_place:
                return saved_locations[key]
        return user_place
    return saved_locations.get(choice, user_place)

def check_saved_locations(user_text: str) -> dict:
    extracted = extract_locations_from_text(user_text)
    if "error" in extracted:
        return extracted
    saved_locations = load_saved_locations()
    start_location = match_location_with_ai(extracted.get("start_location", ""), saved_locations, llm)
    end_location = match_location_with_ai(extracted.get("end_location", ""), saved_locations , llm)
    return {
        "start_location": start_location,
        "end_location": end_location,
        "country": extracted.get("country", "الأردن")
    }

def resolve_address_to_coordinates(address: str) -> str:
    try:
        geocode_result = gmaps.geocode(address)
        if not geocode_result:
            return address
        location = geocode_result[0]['geometry']['location']
        return f"{location['lat']}, {location['lng']}"
    except Exception as e:
        print(f"[خطأ] تعذر تحويل العنوان إلى إحداثيات: {e}")
        return address

def get_location_name_from_coordinates(latlng: str) -> str:
    saved_locations = load_saved_locations()
    for name, value in saved_locations.items():
        if value.strip() == latlng.strip():
            return name
    return latlng

def is_latlng(value: str) -> bool:
    return re.match(r'^\s*\d+(\.\d+)?\s*,\s*\d+(\.\d+)?\s*$', value) is not None

def parse_latlng(latlng_str):
    lat, lng = [float(x.strip()) for x in latlng_str.split(",")]
    return {"lat": lat, "lng": lng}

# 5. الحسابات والخدمات

def calculate_estimated_cost(distance: str, duration: str, car_type: str = "سيارة عادية") -> float:
    if "km" in distance:
        distance_km_value = float(distance.replace('km', '').strip())
    elif "m" in distance:
        distance_km_value = float(distance.replace('m', '').strip()) / 1000
    else:
        distance_km_value = 0
    h, m = 0, 0
    if "hour" in duration or "ساعة" in duration:
        hours = re.search(r'(\d+)\s*(ساعة|hour)', duration)
        mins = re.search(r'(\d+)\s*(دقيقة|min)', duration)
        h = int(hours.group(1)) if hours else 0
        m = int(mins.group(1)) if mins else 0
    else:
        m = int(re.search(r'(\d+)', duration).group(1)) if re.search(r'(\d+)', duration) else 0
    duration_min_value = h * 60 + m
    base_fare = 0.5
    cost = base_fare + (distance_km_value * 0.25) + (duration_min_value * 0.05)
    car_type_multipliers = {
        "سيارة عادية": 1.0,
        "تاكسي": 1.15,
        "سيارة عائلية": 1.3,
        "VIP": 1.5
    }
    multiplier = car_type_multipliers.get(car_type, 1.0)
    cost *= multiplier
    return round(cost, 2)

def translate_to_arabic(step_en: str) -> str:
    prompt = f"""
ترجم جملة الاتجاه التالية من الإنجليزية إلى اللغة العربية بطريقة واضحة وسليمة:

"{step_en}"

الترجمة:
"""
    try:
        response = llm.invoke(prompt)
        return response.content.strip()
    except Exception as e:
        return step_en

def extract_arabic_steps(steps_raw) -> str:
    steps = []
    for step in steps_raw:
        instruction = step['html_instructions']
        clean_instruction = re.sub(r'<[^>]*>', '', instruction)
        arabic_step = translate_to_arabic(clean_instruction)
        steps.append(f"- {arabic_step}")
    return "الخطوات:\n" + "\n".join(steps)

def car_type_selector(_):
    print("🚗 ما نوع السيارة التي تفضلها؟")
    print("الخيارات: [1] عادية، [2] تاكسي، [3] عائلية، [4] VIP")
    choice = input("اكتب نوع السيارة (مثلاً: تاكسي): ").strip().lower()
    if "عادية" in choice:
        return "car_type: عادية (4 ركاب)"
    elif "تاكسي" in choice:
        return "car_type: تاكسي"
    elif "عائلية" in choice or "7" in choice:
        return "car_type: عائلية (7 ركاب)"
    elif "vip" in choice.lower():
        return "car_type: VIP"
    else:
        return "نوع السيارة غير معروف، الرجاء اختيار أحد الأنواع المحددة."

def snap_to_road(lat, lng, api_key):
    try:
        url = f"https://roads.googleapis.com/v1/snapToRoads?path={lat},{lng}&key={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            snapped_points = response.json().get("snappedPoints")
            if snapped_points:
                location = snapped_points[0]["location"]
                return location["latitude"], location["longitude"]
    except Exception as e:
        print(f"[تحذير] تعذر استخدام Google Roads API: {e}")
    return lat, lng

def generate_driver_location(user_location: dict, car_type: str) -> dict:
    user_point = Point(user_location["lat"], user_location["lng"])
    random_distance_m = random.randint(100, 300)
    random_bearing = random.uniform(0, 360)
    driver_point = geopy_distance(meters=random_distance_m).destination(user_point, random_bearing)
    lat, lng = driver_point.latitude, driver_point.longitude
    lat, lng = snap_to_road(lat, lng, GOOGLE_API_KEY)
    estimated_arrival_minutes = (random_distance_m / 100) * 0.5
    return {
        "lat": lat,
        "lng": lng,
        "car_type": car_type,
        "distance_m": random_distance_m,
        "arrival_time_min": round(estimated_arrival_minutes, 1)
    }

def get_directions_arabic(user_text: str, show_steps: bool = False) -> dict:
    locations = check_saved_locations(user_text)
    if "error" in locations:
        return {"error": locations["error"]}
    start_location = locations["start_location"]
    end_location = locations["end_location"]
    country = locations["country"]
    if not start_location or not end_location:
        return {"error": "لم يتم تحديد نقطة البداية أو الوجهة بشكل صحيح."}
    display_start = get_location_name_from_coordinates(start_location)
    display_end = get_location_name_from_coordinates(end_location)
    start_coords = start_location if is_latlng(start_location) else resolve_address_to_coordinates(start_location)
    end_coords = end_location if is_latlng(end_location) else resolve_address_to_coordinates(end_location)
    if isinstance(start_coords, str) and is_latlng(start_coords):
        start_coords = parse_latlng(start_coords)
    if isinstance(end_coords, str) and is_latlng(end_coords):
        end_coords = parse_latlng(end_coords)
    try:
        directions_result = gmaps.directions(
            origin=f"{start_coords['lat']},{start_coords['lng']}",
            destination=f"{end_coords['lat']},{end_coords['lng']}",
            mode="driving",
            language="en"
        )
        if not directions_result:
            return {"error": "لم يتم العثور على مسار بين النقطتين."}
        leg = directions_result[0]['legs'][0]
        distance = leg['distance']['text']
        duration = leg['duration']['text']
        estimated_cost = calculate_estimated_cost(distance, duration)
        return {
            "start_coords": start_coords,
            "end_coords": end_coords,
            "display_start": display_start,
            "display_end": display_end,
            "distance": distance,
            "duration": duration,
            "estimated_cost": estimated_cost,
            "steps": extract_arabic_steps(leg['steps']) if show_steps else None
        }
    except Exception as e:
        return {"error": f"حدث خطأ أثناء جلب الاتجاهات: {e}"}

def plot_trip_map(user_location, driver_location, destination_location, user_name="الراكب", driver_name="السائق"):
    import polyline
    map_center = [user_location["lat"], user_location["lng"]]
    trip_map = folium.Map(location=map_center, zoom_start=14)
    folium.Marker(
        [user_location["lat"], user_location["lng"]],
        tooltip="موقع المستخدم",
        popup=user_name,
        icon=Icon(color="blue", icon="user")
    ).add_to(trip_map)
    folium.Marker(
        [driver_location["lat"], driver_location["lng"]],
        tooltip=f"موقع السائق - {driver_location['car_type']}",
        popup=f"{driver_name} - يبعد {driver_location['distance_m']} متر",
        icon=Icon(color="green", icon="car", prefix="fa")
    ).add_to(trip_map)
    folium.Marker(
        [destination_location["lat"], destination_location["lng"]],
        tooltip="الوجهة",
        popup="الهدف المطلوب",
        icon=Icon(color="red", icon="flag")
    ).add_to(trip_map)
    try:
        driver_to_user = gmaps.directions(
            origin=f"{driver_location['lat']},{driver_location['lng']}",
            destination=f"{user_location['lat']},{user_location['lng']}",
            mode="driving",
            language="en"
        )
        if driver_to_user:
            points = driver_to_user[0]['overview_polyline']['points']
            coords = polyline.decode(points)
            folium.PolyLine(coords, color="orange", weight=5, opacity=0.8, tooltip="مسار السائق").add_to(trip_map)
    except Exception as e:
        print(f"[تحذير] تعذر رسم مسار السائق: {e}")
    try:
        user_to_dest = gmaps.directions(
            origin=f"{user_location['lat']},{user_location['lng']}",
            destination=f"{destination_location['lat']},{destination_location['lng']}",
            mode="driving",
            language="en"
        )
        if user_to_dest:
            points = user_to_dest[0]['overview_polyline']['points']
            coords = polyline.decode(points)
            folium.PolyLine(coords, color="purple", weight=5, opacity=0.8, tooltip="مسار الرحلة").add_to(trip_map)
    except Exception as e:
        print(f"[تحذير] تعذر رسم مسار الرحلة: {e}")
    return trip_map

def live_driver_tracker(driver_info, user_location):
    import polyline
    directions = gmaps.directions(
        origin=f"{driver_info['lat']},{driver_info['lng']}",
        destination=f"{user_location['lat']},{user_location['lng']}",
        mode="driving",
        language="en"
    )
    if not directions:
        print("تعذر إيجاد مسار بين السائق والمستخدم.")
        return
    points = directions[0]['overview_polyline']['points']
    path_coords = polyline.decode(points)
    steps = len(path_coords)
    for i, (lat, lng) in enumerate(path_coords):
        driver_step = driver_info.copy()
        driver_step['lat'] = lat
        driver_step['lng'] = lng
        trip_map = plot_trip_map(
            user_location=user_location,
            driver_location=driver_step,
            destination_location=user_location
        )
        display(trip_map)
        print(f"🚗 السائق يقترب... ({i+1}/{steps})")
        time.sleep(1)
        clear_output(wait=True)
    print("🚗 السائق وصل إلى موقعك!")

def deploy_trip_info(direction_info: dict, car_type: str) -> str:
    if "error" in direction_info:
        return direction_info["error"]

    # استخراج بيانات المستخدم والوجهة
    start_name = direction_info["display_start"]
    end_name = direction_info["display_end"]
    start_coords = direction_info["start_coords"]
    end_coords = direction_info["end_coords"]
    duration = direction_info["duration"]
    estimated_cost = direction_info["estimated_cost"]
    distance = direction_info["distance"]

    # توليد بيانات السائق
    driver_info = generate_driver_location(start_coords, car_type)

    # رسم الخريطة
    trip_map = plot_trip_map(
        user_location=start_coords,
        driver_location=driver_info,
        destination_location=end_coords
    )
    display(trip_map)

    return (
        f"✅ ملخص الرحلة:\n"
        f"🚗 نوع السيارة: {car_type}\n"
        f"📍 موقع الانطلاق: {start_name}\n"
        f"🎯 الوجهة: {end_name}\n"
        f"📏 المسافة: {distance}\n"
        f"🧍‍♂️ السائق يبعد {driver_info['distance_m']} متر\n"
        f"⏱️ سيصل خلال {driver_info['arrival_time_min']} دقيقة تقريبًا\n"
        f"🕒 مدة الرحلة: {duration}\n"
        f"💰 التكلفة التقديرية: {estimated_cost} د.أ\n"
    )

def should_end_conversation(user_input: str) -> bool:
    prompt = f"""
المستخدم كتب: "{user_input}"
هل هو يحاول إنهاء المحادثة؟ جاوب فقط بـ "نعم" أو "لا".
"""
    try:
        response = llm.invoke(prompt).content.strip().lower()
        return "نعم" in response
    except Exception:
        return False


def is_trip_info_complete(direction_data):
    """
    تفحص إذا كانت جميع معلومات الرحلة الأساسية متوفرة وصحيحة.
    """
    required_keys = ["start_coords", "end_coords", "display_start", "display_end", "distance", "duration", "estimated_cost"]
    if not isinstance(direction_data, dict):
        return False
    if "error" in direction_data:
        return False
    for key in required_keys:
        if key not in direction_data or not direction_data[key]:
            return False
    return True
# 8. إنشاء الأدوات والـ Agent

tools = [
    Tool(
        name="get_directions_arabic",
        func=get_directions_arabic,
        description=(
            " غالبا يجب ان يتم استخدام هذه الاداة في اول مراحل تشغيل البرنامج ،تقوم هذه الأداة بحساب اتجاهات القيادة بين نقطتين في الأردن بناءً على وصف المستخدم باللغة العامية. مثل تطبيق Uber"
            "تأخذ النص، تستخرج الموقع الحالي والوجهة، وتحسب المسافة والوقت والتكلفة التقديرية، وتعرض النتيجة بالعربية."
        )
    ),
    Tool(
        name="car_type_selector",
        func=car_type_selector,
         description=(
            "غالبا هذه الاداه الثانية التي سيتم استخدامها بعد اداة (get_directions_arabic) و تُستخدم هذه الأداة  لسؤال المستخدم عن نوع السيارة التي يفضلها عند طلب رحلة. "
            "تشمل الأنواع: عادية (4 ركاب)، تاكسي، سيارة عائلية (7 ركاب)، وVIP. "
            "استخدمها بعد معرفة نقطة الانطلاق والوجهة لتخصيص نوع سيارة الرحلة بدقة."
        )
    )
]

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)  

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)



In [7]:
# 9. تجربة
print("👋 أهلًا وسهلًا بك! أنا مساعدك الذكي للمواصلات في الأردن 🇯🇴")
print("اكتب لي أين أنت الآن وإلى أين تريد الذهاب.\n")
print("اكتب لي (انهي لمحادثة لانهائها).\n")

car_type = "عادية"
while True:
    user_input = input("أنت: ")
    if should_end_conversation(user_input):
        print("👋 شكرًا لاستخدامك المساعد الذكي للمواصلات. يومك سعيد!")
        break
    direction_data = get_directions_arabic(user_input)
    if is_trip_info_complete(direction_data):
        print(deploy_trip_info(direction_data, car_type))
    else:
        response = agent.run(user_input)
        print("🤖", response, "\n")

👋 أهلًا وسهلًا بك! أنا مساعدك الذكي للمواصلات في الأردن 🇯🇴
اكتب لي أين أنت الآن وإلى أين تريد الذهاب.

اكتب لي (انهي لمحادثة لانهائها).



✅ ملخص الرحلة:
🚗 نوع السيارة: عادية
📍 موقع الانطلاق: الدار
🎯 الوجهة: الرفيد
📏 المسافة: 11.5 km
🧍‍♂️ السائق يبعد 282 متر
⏱️ سيصل خلال 1.4 دقيقة تقريبًا
🕒 مدة الرحلة: 19 mins
💰 التكلفة التقديرية: 4.33 د.أ



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: العفو! إذا كان لديك أي سؤال أو تحتاج إلى مساعدة في شيء آخر، فلا تتردد في طرحه. أنا هنا للمساعدة!
```

> Finished chain.
🤖 العفو! إذا كان لديك أي سؤال أو تحتاج إلى مساعدة في شيء آخر، فلا تتردد في طرحه. أنا هنا للمساعدة!
``` 

```
Thought: Do I need to use a tool? No
AI: العفو! إذا كان لديك أي سؤال أو تحتاج إلى مساعدة في شيء آخر، فلا تتردد في طرحه. أنا هنا للمساعدة!
```

> Finished chain.
🤖 العفو! إذا كان لديك أي سؤال أو تحتاج إلى مساعدة في شيء آخر، فلا تتردد في طرحه. أنا هنا للمساعدة!
``` 

👋 شكرًا لاستخدامك المساعد الذكي للمواصلات. يومك سعيد!
👋 شكرًا لاستخدامك المساعد الذكي للمواصلات. يومك سعيد!
